# Drag Control Statespace with Kalman Filter

### Importing Libraries

In [92]:
import numpy as np
import sympy as sym
import json
import matplotlib.pyplot as plt
from scipy import linalg
from scipy.interpolate import interp1d
from sympy import Matrix

### States

In [93]:
# Altitude
x = sym.symbols('x')
# Vertical Velocity
u = sym.symbols('u')

### Inputs

In [94]:
#Flap Extension Length
L = sym.symbols('L')

### Constants

In [95]:
g, W_f, Ixx, Sref, rho, m, D, Cd = sym.symbols('g, W_f, Ixx, Sref, rho, m, D, Cd')

### State Space Model

In [96]:
# EOM
F = -((rho* (u**2) * Sref * Cd) / 2) - (m*g)
# F = -m*g
accel = F / m

# Xdot Vector
hdot = u
hddot = accel
f_sym = sym.Matrix.vstack(sym.Matrix([[hdot],[hddot]]))

# States
s = [x,u]
# Inputs
i = [L]
# Params
params = [g, W_f, Ixx, Sref, rho, m, D, Cd]

f_sym

Matrix([
[                            u],
[(-Cd*Sref*rho*u**2/2 - g*m)/m]])

### Turning f into a function

In [97]:
f = sym.lambdify(s + i + params, f_sym)

m = 21.364                 # dry mass of the rocket 
D = 0.1056132               # rocket diameter       
Sref = sym.pi * (D/2)**2    # aerodynamic reference area in m^2
W_flap = .03175
rho = 1.225             #! Not constant throughout flight
g = 9.81                #! Not constant throughout flight
Cd = .58                #! Not constant throughout flight


### Equilibrium Values

In [98]:
x_e = 9144 #m
u_e = 0.1 #m/s
L_e = 0.0254 # m

s_eq = [x_e,u_e]
i_eq = [L_e]
params_eq = [g, W_f, Ixx, Sref, rho, m, D, Cd]

f_eq = f(*s_eq,*i_eq,*params_eq)
f_eq


array([[0.1],
       [-9.81 - 4.63690212747936e-7*pi]], dtype=object)

### A Matrix

In [99]:
A_sym = f_sym.jacobian(s)
A_sym


Matrix([
[0,                1],
[0, -Cd*Sref*rho*u/m]])

In [100]:
A_num = sym.lambdify(s+i+params, A_sym)
A = A_num(*s_eq,*i_eq,*params_eq)
A

array([[0, 1],
       [0, -9.27380425495872e-6*pi]], dtype=object)

### B Matrix

In [101]:
# Need to add flap equations into EOMs before doing this
B_sym = f_sym.jacobian(i)
B_sym


Matrix([
[0],
[0]])

In [102]:
B_num = sym.lambdify(s+i+params,B_sym)
B = B_num(*s_eq,*i_eq,*params_eq)
B

array([[0],
       [0]])

### Discretization

Converting Continous State Matrix to Discrete 

$ F =  e^{(A \Delta t)}$

$\Delta t$ is the time step between measurements